# Arrays and views

Julia has excellent functionality for manipulating $N$-dimensional arrays. We will have a quick look at this subject (which is more complicated than you might suspect). 

Note that in Julia 0.7 there are significant changes, especially to how transposes work, and how different types of manipulations of arrays look.

Let's define a $3 \times 3$ array (matrix):

In [1]:
M = [1 2 3; 4 5 6; 7 8 9]  # a 3x3 matrix

3×3 Array{Int64,2}:
 1  2  3
 4  5  6
 7  8  9

In [2]:
M2 = zeros(10, 10)

10×10 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [ ]:
typeof(M)

We can extract part of the matrix using indexing notation:

In [3]:
part = M[2:3, 1:2]

2×2 Array{Int64,2}:
 4  5
 7  8

What happens if we modify `part`?

In [4]:
part[1, 1]

4

In [5]:
part[1, 1] = 10

10

In [6]:
part

2×2 Array{Int64,2}:
 10  5
  7  8

In [7]:
M

3×3 Array{Int64,2}:
 1  2  3
 4  5  6
 7  8  9

We see that `M` has *not* been modified; that means that `part` was a **copy** of that part of `M`.

## Views

We often do *not* want a copy, but rather just a reference to the same data, which is called a `view`: 

In [8]:
V = view(M, 2:3, 1:2)

2×2 SubArray{Int64,2,Array{Int64,2},Tuple{UnitRange{Int64},UnitRange{Int64}},false}:
 4  5
 7  8

In [ ]:
typeof(V)

Although this type looks (and is) somewhat complicated, it just contains the necessary information for the object to manipulate correctly the underlying data. To decompose this type expression, see the array types notebook.

In [ ]:
V isa AbstractArray

In [ ]:
subtypes(AbstractArray)  # behave like a generalized Array

If we modify `V`, then `M` also gets modified, since it is the same data:

In [ ]:
V

In [ ]:
show(V)

In [9]:
V[1, 1]

4

In [10]:
V

2×2 SubArray{Int64,2,Array{Int64,2},Tuple{UnitRange{Int64},UnitRange{Int64}},false}:
 4  5
 7  8

In [11]:
V[1, 1] = 100

100

In [12]:
V

2×2 SubArray{Int64,2,Array{Int64,2},Tuple{UnitRange{Int64},UnitRange{Int64}},false}:
 100  5
   7  8

In [13]:
M

3×3 Array{Int64,2}:
   1  2  3
 100  5  6
   7  8  9

If we have a complicated expression, we can use the `@view` macro to make the syntax nicer; this turns an indexing operation into the corresponding `view`:

In [14]:
V2 = @view M[2:3, 1:2]

2×2 SubArray{Int64,2,Array{Int64,2},Tuple{UnitRange{Int64},UnitRange{Int64}},false}:
 100  5
   7  8

In [15]:
V2[2, 1] = -1

-1

In [16]:
M

3×3 Array{Int64,2}:
   1  2  3
 100  5  6
  -1  8  9

To turn every indexing operation in a block of code into a view, use `@views`.

## In-place and vectorized operations: "`.`" ("pointwise")

Suppose we have two matrices and wish to add one to the other:

In [17]:
N = 10
A = rand(N, N)
B = rand(N, N);

In [18]:
B

10×10 Array{Float64,2}:
 0.940558   0.511221     0.589263     …  0.733864  0.288038   0.278856
 0.71302    0.000712072  0.071074        0.238767  0.1266     0.658453
 0.857103   0.582837     0.000572778     0.336562  0.262184   0.487623
 0.0999008  0.234913     0.0517096       0.108577  0.358644   0.706009
 0.897085   0.830347     0.520947        0.199525  0.261288   0.827817
 0.416307   0.149185     0.87108      …  0.806895  0.0271634  0.916133
 0.65622    0.541364     0.530482        0.417788  0.211856   0.692344
 0.537186   0.794077     0.55571         0.79291   0.881025   0.634474
 0.71468    0.291606     0.651847        0.264406  0.288952   0.514322
 0.928955   0.434726     0.917389        0.161331  0.603198   0.722057

Coming from other languages, we might expect to be able to write `A += B`, and indeed this works:

In [19]:
A += B   # A = A + B

10×10 Array{Float64,2}:
 1.93906   0.69086   1.48187   0.892602  …  1.38183   1.16699    0.898903
 1.20256   0.704084  0.423812  1.65872      0.849729  1.06365    0.814794
 1.38007   0.754468  0.122831  1.23566      0.375462  0.816591   1.04813 
 0.730539  0.314742  0.626422  1.27637      0.917084  0.420845   1.55054 
 1.31825   1.53193   0.833262  1.25407      1.10284   0.329601   1.29482 
 1.30103   1.09783   1.75681   0.404778  …  1.13973   0.0990398  1.47171 
 1.36414   1.10006   1.05532   1.50767      1.33133   0.290842   1.32058 
 0.594179  1.51431   0.643904  0.827423     1.52021   1.48067    1.44417 
 0.871395  0.307144  0.826177  0.928432     1.03496   0.57245    1.43347 
 1.8059    0.732801  1.26973   0.879367     0.597694  1.26294    0.910311

In [20]:
A

10×10 Array{Float64,2}:
 1.93906   0.69086   1.48187   0.892602  …  1.38183   1.16699    0.898903
 1.20256   0.704084  0.423812  1.65872      0.849729  1.06365    0.814794
 1.38007   0.754468  0.122831  1.23566      0.375462  0.816591   1.04813 
 0.730539  0.314742  0.626422  1.27637      0.917084  0.420845   1.55054 
 1.31825   1.53193   0.833262  1.25407      1.10284   0.329601   1.29482 
 1.30103   1.09783   1.75681   0.404778  …  1.13973   0.0990398  1.47171 
 1.36414   1.10006   1.05532   1.50767      1.33133   0.290842   1.32058 
 0.594179  1.51431   0.643904  0.827423     1.52021   1.48067    1.44417 
 0.871395  0.307144  0.826177  0.928432     1.03496   0.57245    1.43347 
 1.8059    0.732801  1.26973   0.879367     0.597694  1.26294    0.910311

In [ ]:
expand(:(A += B))

We see that this is just "syntactic sugar" (i.e. a cute way of writing) `A = A + B`.

However, it turns out that this does not do what you might think it does, namely "in-place addition", in which each element of `A` is updated in place. Rather, it allocates a new temporary object for the result of `A + B`. We can see this:

In [21]:
using BenchmarkTools

N = 1000
A = rand(N, N)
B = rand(N, N)

@btime $A += $B;

INFO: Recompiling stale cache file /Users/dpsanders/.julia/lib/v0.6/BenchmarkTools.ji for module BenchmarkTools.


  1.748 ms (2 allocations: 7.63 MiB)


Note the large amount of allocation here (1,000,000 $\times$ 8 bytes).

The in-place behaviour can be obtained using **pointwise operators** (with `.`); this is called **broadcasting**:

In [22]:
A .= A .+ B

1000×1000 Array{Float64,2}:
 1.16981   0.936588  1.7159    1.26022   …  0.745496  1.19038   1.2424  
 1.63981   1.7334    1.21014   0.813907     1.03248   1.58853   0.363102
 0.497984  1.02264   1.08602   1.30959      0.316367  1.09023   1.21523 
 0.867906  0.898176  1.44549   1.41544      0.721673  0.624086  0.12361 
 0.399844  1.09694   0.972533  1.01094      1.4194    1.14485   1.04909 
 0.903221  0.848083  1.08933   1.26613   …  0.414192  0.291594  0.936757
 1.20433   1.35424   0.214067  0.758506     1.01577   1.77321   0.287125
 1.18217   0.933562  1.06104   1.36909      1.08343   1.25878   0.767985
 1.39354   0.924212  1.61625   0.871726     0.659069  0.947622  1.59316 
 0.320187  1.69944   1.25073   0.32946      0.65151   0.816842  1.31343 
 1.17052   0.983742  0.903218  1.84967   …  0.754539  0.252915  0.247722
 0.876398  0.114439  1.1105    0.740464     0.768053  0.730706  0.707669
 0.654488  1.40102   0.569995  1.07993      1.24717   0.996053  1.43778 
 ⋮                     

In [23]:
@btime $A .= $A .+ $B;  # no allocations

  714.579 μs (0 allocations: 0 bytes)


Furthermore, we can chain such operations together with no creation of temporaries:

In [ ]:
C = rand(1000, 1000)

@btime A .+= B + C;  # allocates

In [ ]:
@btime $A .+= $B .+ $C  # does not allocate  

This is equivalent to

In [ ]:
for i in eachindex(A)
    A[i] += B[i] + C[i]
end

See [this blog post by Steven Johnson](https://julialang.org/blog/2017/01/moredots) for more details.

There is a `@.` macro for "pointing" every operation:

In [26]:
C = rand(N, N);

In [27]:
@. A = B + C

1000×1000 Array{Float64,2}:
 1.12993   0.948976  1.39354   1.56599   …  0.323939  1.22776   1.3624  
 1.53569   1.48761   1.69706   0.985504     1.04491   1.45091   0.39966 
 0.69185   0.725004  0.880922  0.66143      0.925565  1.25837   0.385692
 0.664166  0.606125  1.47966   0.948206     0.706564  0.39955   0.641969
 0.245766  0.505511  0.926429  0.857936     1.78711   1.5214    1.34055 
 1.20722   1.15743   0.760931  1.48047   …  0.991465  0.83315   0.883675
 0.860133  1.1847    0.852797  1.03532      1.08959   1.26387   0.713851
 1.13657   0.571965  1.18769   1.03933      1.72771   0.826335  0.929174
 1.03179   1.39106   1.68071   1.54826      1.1306    0.316485  1.49711 
 1.1897    1.24612   0.961993  0.35677      0.58992   0.779688  1.27124 
 1.35833   0.766976  1.45629   1.75238   …  0.131441  0.229206  0.259513
 0.969386  0.660069  1.16434   1.49924      1.17088   0.908893  0.848153
 0.786586  0.999362  1.02491   0.768294     1.25841   1.09207   1.67478 
 ⋮                     

In [28]:
v = [1, 2, 3]

3-element Array{Int64,1}:
 1
 2
 3

In [29]:
f(x) = x^2 + 2

f (generic function with 1 method)

In [30]:
[f(x) for x in v]

3-element Array{Int64,1}:
  3
  6
 11

In [31]:
map(f, v)

3-element Array{Int64,1}:
  3
  6
 11

In [35]:
f.(v)

3-element Array{Int64,1}:
  3
  6
 11

In [36]:
g(x) = exp(x)

g (generic function with 1 method)

In [37]:
g.(f.(v))

3-element Array{Float64,1}:
    20.0855
   403.429 
 59874.1   

In [41]:
h(v) = g.(f.(v))

h (generic function with 1 method)

In [42]:
@code_warntype h(v)

Variables:
  #self# <optimized out>
  v::Array{Int64,1}
  #11 <optimized out>
  T <optimized out>
  shape <optimized out>
  iter <optimized out>
  C::Array{Float64,1}
  keeps@_8::Tuple{Tuple{Bool}}
  Idefaults@_9::Tuple{Tuple{Int64}}
  #temp#@_10 <optimized out>
  keeps@_11 <optimized out>
  Idefaults@_12 <optimized out>
  #temp#@_13 <optimized out>
  keep@_14::Tuple{Bool}
  Idefault@_15::Tuple{Int64}
  #temp#@_16 <optimized out>
  ind1 <optimized out>
  keep@_18 <optimized out>
  Idefault@_19 <optimized out>
  #temp#@_20 <optimized out>
  I_1 <optimized out>
  val_1::Int64
  result::Float64
  I@_24 <optimized out>
  i#705::Int64
  I@_26 <optimized out>
  n#704::Int64
  i#703 <optimized out>
  #temp#@_29::Bool
  r#702 <optimized out>
  A_1 <optimized out>
  keep_1::Tuple{Bool}
  Idefault_1::Tuple{Int64}
  #temp#@_34::Int64
  #temp#@_35 <optimized out>

Body:
  begin 
      $(Expr(:inbounds, false))
      # meta: location broadcast.jl broadcast 455
      # meta: location broadcast.jl br

@btime @. $A += $B + $C

## Efficient small matrices and vectors

For small matrices and vectors, the generic vector and matrix code is too slow, since the type does not contain the information on the number of elements contained in the array, so that generic loops are used.

The `StaticArrays.jl` package fixes this problem by **unrolling** operations for small arrays.

In [43]:
# Pkg.add("StaticArrays")

using StaticArrays, BenchmarkTools

In [45]:
v = SVector(1, 2)

2-element StaticArrays.SArray{Tuple{2},Int64,1,2}:
 1
 2

In [46]:
function bench()
    x = SVector(1, 2)
    y = [1, 2]
    
    @btime $x + $x
    @btime $y + $y
end

bench (generic function with 1 method)

In [47]:
bench()

  1.909 ns (0 allocations: 0 bytes)
  37.175 ns (1 allocation: 96 bytes)


2-element Array{Int64,1}:
 2
 4

In [48]:
bench()

  1.432 ns (0 allocations: 0 bytes)
  37.145 ns (1 allocation: 96 bytes)


2-element Array{Int64,1}:
 2
 4

In [49]:
x = SVector(1, 2)
@code_lowered x + x

CodeInfo(:(begin 
        nothing
        nothing
        return (StaticArrays.map)(StaticArrays.+, a, b)
    end))

In [50]:
@code_typed x + x

CodeInfo(:(begin 
        SSAValue(4) = a
        SSAValue(5) = b
        $(Expr(:inbounds, false))
        # meta: location /Users/dpsanders/.julia/v0.6/StaticArrays/src/mapreduce.jl map 11
        SSAValue(2) = SSAValue(4)
        SSAValue(3) = SSAValue(5)
        # meta: location /Users/dpsanders/.julia/v0.6/StaticArrays/src/mapreduce.jl _map 20
        # meta: location /Users/dpsanders/.julia/v0.6/StaticArrays/src/mapreduce.jl # line 29:
        $(Expr(:inbounds, true))
        #temp# = $(Expr(:new, StaticArrays.SArray{Tuple{2},Int64,1,2}, :((StaticArrays.tuple)((Base.add_int)((Base.getfield)((Core.getfield)(SSAValue(2), :data)::Tuple{Int64,Int64}, 1)::Int64, (Base.getfield)((Core.getfield)(SSAValue(3), :data)::Tuple{Int64,Int64}, 1)::Int64)::Int64, (Base.add_int)((Base.getfield)((Core.getfield)(SSAValue(2), :data)::Tuple{Int64,Int64}, 2)::Int64, (Base.getfield)((Core.getfield)(SSAValue(3), :data)::Tuple{Int64,Int64}, 2)::Int64)::Int64)::Tuple{Int64,Int64})))
        goto 15
      

In [51]:
@code_llvm x + x


define void @"julia_+_64099"(%SArray* noalias nocapture sret, %SArray* nocapture readonly dereferenceable(16), %SArray* nocapture readonly dereferenceable(16)) #0 !dbg !5 {
top:
  %3 = getelementptr inbounds %SArray, %SArray* %1, i64 0, i32 0, i64 0
  %4 = getelementptr inbounds %SArray, %SArray* %2, i64 0, i32 0, i64 0
  %5 = load i64, i64* %3, align 8
  %6 = load i64, i64* %4, align 8
  %7 = add i64 %6, %5
  %8 = getelementptr inbounds %SArray, %SArray* %1, i64 0, i32 0, i64 1
  %9 = getelementptr inbounds %SArray, %SArray* %2, i64 0, i32 0, i64 1
  %10 = load i64, i64* %8, align 8
  %11 = load i64, i64* %9, align 8
  %12 = add i64 %11, %10
  %"#temp#.sroa.0.sroa.0.0.#temp#.sroa.0.0..sroa_cast1.sroa_idx" = getelementptr inbounds %SArray, %SArray* %0, i64 0, i32 0, i64 0
  store i64 %7, i64* %"#temp#.sroa.0.sroa.0.0.#temp#.sroa.0.0..sroa_cast1.sroa_idx", align 8
  %"#temp#.sroa.0.sroa.2.0.#temp#.sroa.0.0..sroa_cast1.sroa_idx7" = getelementptr inbounds %SArray, %SArray* %0, i64 0, i32

In [52]:
@code_native x + x

	.section	__TEXT,__text,regular,pure_instructions
Filename: linalg.jl
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 29
	movq	(%rdx), %rax
	movq	8(%rdx), %rcx
	addq	(%rsi), %rax
	addq	8(%rsi), %rcx
Source line: 10
	movq	%rax, (%rdi)
	movq	%rcx, 8(%rdi)
	movq	%rdi, %rax
	popq	%rbp
	retq
	nop


In [53]:
y = [1, 2]
@code_native y + y

	.section	__TEXT,__text,regular,pure_instructions
Filename: arraymath.jl
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r12
	pushq	%rbx
	subq	$64, %rsp
	movq	%rsi, %r12
	movq	%rdi, %r15
	movabsq	$jl_get_ptls_states_fast, %rax
	callq	*%rax
	movq	%rax, %r14
	movq	$0, -40(%rbp)
	movq	$0, -48(%rbp)
	movq	$4, -64(%rbp)
	movq	(%r14), %rax
	movq	%rax, -56(%rbp)
	leaq	-64(%rbp), %rax
	movq	%rax, (%r14)
Source line: 64
	movq	24(%r15), %rax
Source line: 64
	movq	24(%r12), %rcx
	xorl	%ebx, %ebx
Source line: 38
	testq	%rax, %rax
	cmovsq	%rbx, %rax
	movq	%rax, -72(%rbp)
	testq	%rcx, %rcx
	cmovsq	%rbx, %rcx
	movq	%rcx, -80(%rbp)
	movabsq	$promote_shape, %rax
	leaq	-72(%rbp), %rdi
	leaq	-80(%rbp), %rsi
	callq	*%rax
Source line: 64
	movq	24(%r15), %rax
Source line: 64
	movq	24(%r12), %rcx
Source line: 63
	testq	%rax, %rax
	cmovsq	%rbx, %rax
	movq	%rax, -88(%rbp)
	testq	%rcx, %rcx
	cmovsq	%rbx, %rcx
	movq	%rcx, -96(%rbp)
	movabsq	$_bcs1, %rax
	leaq	-88(%rbp), %rdi
	leaq	-96(%rbp), %rsi
	c